<a href="https://colab.research.google.com/github/khanfs/ComputationalBiology-xGenomics/blob/main/REACTOME_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reactome API**

REACTOME is an open-source, open access, manually curated and peer-reviewed pathway database. Provides bioinformatics tools for the visualization, interpretation and analysis of pathway knowledge.

Interaction, reaction and pathway data are provided as downloadable flat, Neo4j GraphDB, MySQL, BioPAX, SBML and PSI-MITAB files and are also accessible through our Web Services APIs. Software and instructions for local installation of the Reactome database, website, and data entry tools are also available to support independent pathway curation.

**Resources**:

* [Reactome Python Package](https://github.com/reactome/reactome2py)

In [1]:
! pip install reactome2py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for json5: filename=json5-0.9.8-py2.py3-none-any.whl size=18604 sha256=8f3d1742418d7155c85a09a7e6800afb6b5c39ec440a3e20d61a5436956c620f
  Stored in directory: /root/.cache/pip/wheels/ac/9b/de/6e4fd8f159d3dfa42c42ceddf2184fda29ea7fb1e8f5f8371c
Successfully built json5


##**Example: Assessing pathways and drug targets in Cyclic Multiplexed-Immunofluorescence (cmIF) image biomarkers** 

Immunotherapy harnesses the power of the adaptive immune system and has revolutionized the field of oncotherapy, as novel therapeutic strategies have been introduced into clinical use. The development of immune checkpoint inhibitors has led to durable control of disease in a subset of advanced cancer patients, such as those with melanoma and non-small cell lung cancer. However, predicting patient responses to therapy remains a major challenge, due to the remarkable genomic, epigenetic, and microenvironmental heterogeneity present in each tumor. 


**Reference**: Eng J, Thibault G, Luoh SW, Gray JW, Chang YH, Chin K. Cyclic Multiplexed-Immunofluorescence (cmIF), a Highly Multiplexed Method for Single-Cell Analysis. Methods Mol Biol. 2020;2055:521-562. [doi: 10.1007/978-1-4939-9773-2_24](https://pubmed.ncbi.nlm.nih.gov/31502168/). PMID: 31502168.

###**1. After mapping proteins to gene names, we wrangle the matrix into a dictionary where each image section would hold the biomarkers as its values.** 

In [2]:
import reactome2py

In [3]:
from reactome2py import analysis, content, utils
from collections import defaultdict
from itertools import chain
from operator import methodcaller
import pandas as pd

In [4]:
path = "/content/marker_matrix_gene.csv"

In [5]:
df = pd.read_csv(path, index_col=0)
df = df.apply(lambda x : x.str.split(","), axis=0)
row_dict = df.T.to_dict('records')

def imggenes2dict(row_dict):
    dd = defaultdict(list)
    dict_items = map(methodcaller('items'), row_dict)
    
    for k, v in chain.from_iterable(dict_items):
        dd[k].extend(v)
    
    img_dict = dict(dd)
    return(img_dict)

img_dict = imggenes2dict(row_dict)

###**2. Now we can pass the biomarkers to fetch all possible reactome pathways available for each image segment.** 

In [6]:
result = [analysis.identifiers(ids=",".join(d)) for d in img_dict.values()]
tokens = [r['summary']['token'] for r in result]


def get_pathways(t):
    token_result = analysis.token(token=t, species='Homo sapiens', page_size='-1', page='-1', sort_by='ENTITIES_FDR', 
                                  order='ASC', resource='TOTAL', p_value='1', include_disease=True, 
                                  min_entities=None, max_entities=None)
    l = [p['name'] for p in token_result['pathways']]
    l.sort() 
    return(l)

pathways = [get_pathways(t) for t in tokens]
pathways[1]

['Adaptive Immune System',
 'Base Excision Repair',
 'Cell Cycle',
 'Cell Cycle, Mitotic',
 'Chromosome Maintenance',
 'Costimulation by the CD28 family',
 'DNA Damage Bypass',
 'DNA Double-Strand Break Repair',
 'DNA Repair',
 'DNA Replication',
 'DNA strand elongation',
 'Disease',
 'Dual Incision in GG-NER',
 'Dual incision in TC-NER',
 'E3 ubiquitin ligases ubiquitinate target proteins',
 'Extension of Telomeres',
 'G0 and Early G1',
 'G1/S Transition',
 'G1/S-Specific Transcription',
 'Gap-filling DNA repair synthesis and ligation in GG-NER',
 'Gap-filling DNA repair synthesis and ligation in TC-NER',
 'Gene expression (Transcription)',
 'Generic Transcription Pathway',
 'Global Genome Nucleotide Excision Repair (GG-NER)',
 'HDR through Homologous Recombination (HRR)',
 'HDR through Homologous Recombination (HRR) or Single Strand Annealing (SSA)',
 'Homology Directed Repair',
 'Immune System',
 'Infectious disease',
 'Innate Immune System',
 'Lagging Strand Synthesis',
 'Leading S

###**3. Given the biomarkers, we can also fetch all available drug targets from Drug Central.**

In [8]:
result_drug_targets = [utils.genelist_drug_target(ids=",".join(d), source="drugcentral") for d in img_dict.values()]
result_drug_targets[1]